<a href="https://colab.research.google.com/github/2320030030/Aiml---Project/blob/main/Sample%20Project%20Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import cv2
import numpy as np
from mtcnn.mtcnn import MTCNN
from tensorflow.keras.models import load_model
from sklearn.preprocessing import Normalizer
import os

# Load the FaceNet model for embedding
model = load_model('facenet_keras.h5')  # Ensure you have the pre-trained model file

# L2 normalization for feature vectors
in_encoder = Normalizer(norm='l2')

# Initialize MTCNN for face detection
detector = MTCNN()

# Sample dataset: Dictionary with image paths and names
registered_faces = {
    "Indira Priyadarshini": "dataset/indira.jpg",
    "Venkat Reddy": "dataset/venkat.jpg",
    "Shanmukha Devi": "dataset/shanmukha.jpg",
    "Keerthi Dhara Reddy": "dataset/keerthi.jpg"
}

# Function to extract face from an image
def extract_face(filename, required_size=(160, 160)):
    img = cv2.imread(filename)
    results = detector.detect_faces(img)
    if len(results) == 0:
        return None
    x1, y1, width, height = results[0]['box']
    x1, y1 = abs(x1), abs(y1)
    x2, y2 = x1 + width, y1 + height
    face = img[y1:y2, x1:x2]
    face = cv2.resize(face, required_size)
    return face

# Convert faces into FaceNet embeddings
def get_embedding(model, face_pixels):
    face_pixels = face_pixels.astype('float32')
    mean, std = face_pixels.mean(), face_pixels.std()
    face_pixels = (face_pixels - mean) / std
    samples = np.expand_dims(face_pixels, axis=0)
    yhat = model.predict(samples)
    return yhat[0]

# Prepare dataset of registered embeddings
def prepare_registered_faces():
    embeddings = {}
    for name, image_path in registered_faces.items():
        face = extract_face(image_path)
        if face is not None:
            embedding = get_embedding(model, face)
            embedding = in_encoder.transform([embedding])[0]  # Normalize embedding
            embeddings[name] = embedding
    return embeddings

# Compare new face embedding with registered embeddings
def recognize_face(embeddings, new_embedding, threshold=0.5):
    min_dist = 100
    identity = None
    for name, embedding in embeddings.items():
        dist = np.linalg.norm(embedding - new_embedding)
        if dist < min_dist and dist < threshold:
            min_dist = dist
            identity = name
    return identity

# Real-time attendance system
def run_attendance_system():
    embeddings = prepare_registered_faces()
    cap = cv2.VideoCapture(0)  # Access webcam
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        faces = detector.detect_faces(frame)
        for result in faces:
            x1, y1, width, height = result['box']
            x1, y1 = abs(x1), abs(y1)
            x2, y2 = x1 + width, y1 + height
            face = frame[y1:y2, x1:x2]
            face = cv2.resize(face, (160, 160))
            embedding = get_embedding(model, face)
            embedding = in_encoder.transform([embedding])[0]
            name = recognize_face(embeddings, embedding)

            if name is not None:
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame, name, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            else:
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 2)
                cv2.putText(frame, "Unknown", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

        cv2.imshow('Attendance System', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == '__main__':
    run_attendance_system()


ModuleNotFoundError: No module named 'mtcnn'